In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Wed Jan 25 12:36:13 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   70C    P0    30W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import os
os.chdir('/content/drive/MyDrive/Study/Data Science/AI 자연어 처리 전문가 양성 과정 6기/Project 3')

In [3]:
!pip install datasets konlpy sentence_transformers transformers
#!pip install git+https://github.com/ssut/py-hanspell.git
#!pip install git+https://github.com/haven-jeon/PyKoSpacing.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
from datasets import Dataset
from IPython.display import display
from konlpy.tag import Okt
#from hanspell import spell_checker
#from pykospacing import Spacing
from nltk.translate.bleu_score import sentence_bleu
from sentence_transformers import SentenceTransformer, util
from tqdm.auto import tqdm
from transformers import AutoTokenizer, pipeline

import torch

import numpy as np
import pandas as pd

In [5]:
df_a = pd.read_csv('output/trans_test_dastset.csv')[['번역전', '한글번역']]
df_a.columns = ['spoken', 'written']

In [6]:
'''def check_spelling(sent):
    spacing = Spacing()
    kospacing_sent = spacing(sent)
    hanspell_sent = spell_checker.check(sent).checked
    return hanspell_sent'''

'def check_spelling(sent):\n    spacing = Spacing()\n    kospacing_sent = spacing(sent)\n    hanspell_sent = spell_checker.check(sent).checked\n    return hanspell_sent'

In [13]:
def filter(df):
    df['spoken'], df['written'] = df['spoken'].str.strip(), df['written'].str.strip()
    #df['written'] = df['written'].map(check_spelling)
    print(df.shape[0])
    
    # 1. 문장 길이를 이용한 필터링
    df = df.assign(
        spoken_length=df['spoken'].map(lambda x: len(x)),
        written_length=df['written'].map(lambda x: len(x))
    )

    mask = (df.spoken_length <= 100) & (df.written_length <= 100) & (df.written_length >= df.spoken_length * 0.9)
    display(df.loc[-mask, :])
    df = df.loc[mask, :]
    print(f'문장 길이를 이용한 필터링 -> {df.shape[0]}')

    # 2. BLEU score를 이용한 필터링
    okt = Okt()
    blue_score = []
    for idx, row in df.iterrows():
        candidate = okt.morphs(row['spoken'])
        references = [row['written']]
        references = [okt.morphs(s) for s in references]
        blue_score.append(sentence_bleu(references, candidate, weights=(1, 0, 0, 0)))
    df['blue_score'] = blue_score

    mask = (df.blue_score != 0) & (df.blue_score != 1)
    display(df.loc[-mask, :])
    df = df.loc[mask, :]
    print(f'BLEU score를 이용한 필터링 -> {df.shape[0]}')

    # 3. KLUE-NLI를 이용한 필터링
    classifier = pipeline(
        "text-classification",
        model="Huffon/klue-roberta-base-nli",
        return_all_scores=True,
        device=0
    )
    tokenizer = AutoTokenizer.from_pretrained("Huffon/klue-roberta-base-nli")
    sep_token = tokenizer.sep_token
    nli_label = []
    nli_score = []
    df['concat'] = df['spoken'].str.cat(df['written'], sep=f' {sep_token} ')
    ds = Dataset.from_pandas(df)['concat']
    for output in tqdm(classifier(ds)):
        output_dict = {i['label']: i['score'] for i in output}
        label = max(output_dict.keys(), key=lambda x: output_dict[x])
        score = max(output_dict.values())
        nli_label.append(label)
        nli_score.append(score)
    df['nli_label'] = nli_label
    df['nli_score'] = nli_score
    df = df.drop(['concat'], axis=1)

    mask = (df.nli_label == 'ENTAILMENT') & (df.nli_score > 0.8)
    display(df.loc[-mask, :])
    df = df.loc[mask, :]
    print(f'KLUE-NLI를 이용한 필터링를 이용한 필터링 -> {df.shape[0]}')

    # 4. 코사인 유사도를 이용한 필터링
    model = SentenceTransformer('ddobokki/klue-roberta-small-nli-sts')
    model.to(torch.device('cuda'))
    df['combine'] = df['spoken'].map(lambda x: [x]) + df['written'].map(lambda x: [x])
    ds = Dataset.from_pandas(df)['combine']
    cosine_scores = []
    for sentences in tqdm(ds):
        embeddings = model.encode(sentences)
        #Compute cosine-similarities
        cosine_scores.append(util.cos_sim(embeddings[0], embeddings[1]))
    df['cosine_scores'] = cosine_scores
    df = df.drop(['combine'], axis=1)

    mask = (df.cosine_scores >= 0.9)
    display(df.loc[-mask, :])
    df = df.loc[mask, :]
    print(f'코사인 유사도를 이용한 필터링 -> {df.shape[0]}')

    df = df.reset_index(drop=True)
    return df

In [14]:
df_b = df_a.head(10)
filter(df_b)

10


<ipython-input-13-264d12fc9d02>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['spoken'], df['written'] = df['spoken'].str.strip(), df['written'].str.strip()


,spoken,written,spoken_length,written_length
0,사과는 잘 씻은 뒤 껍질 채 먹는 게 좋다네요.,사과를 씻고 먹는 것이 낫습니다.,26,18
2,우리가 작성해서 전달해 드려야 하나요?,우리는 쓰고 전달해야합니까?,21,15
4,그는 틀림없이 꽃 가게에 들렸을 겁니다.,그는 꽃 가게에 있었을 것입니다.,22,18
8,"아래 그림의 bga 부분에 대해서는, nsmd에서 smd로 존재하는 것과 같은 방식...",아래 그림의 BGA 부분은 NSMD의 SMD와 같은 방식으로 계속 변경됩니다.,63,43
9,500달러만 당신들에게 요청할게요.,500 달러에만 물을 것입니다.,19,17


문장 길이를 이용한 필터링 -> 5


/usr/local/lib/python3.8/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.8/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.8/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg

...

KLUE-NLI를 이용한 필터링를 이용한 필터링 -> 4


  0%|          | 0/4 [00:00<?, ?it/s]

,spoken,written,spoken_length,written_length,blue_score,nli_label,nli_score,cosine_scores
1,내가 언제까지 거기 가면 됩니까?,거기에 얼마나 오래 갈 수 있습니까?,18,20,0.250000,ENTAILMENT,0.994652,[[tensor(0.5265)]]
5,"그녀가 좋아하는 가수는 박효신인데, 나는 그녀가 그와 같은 멋진 남자친구가 생기길 ...","그녀가 가장 좋아하는 가수는 Park Hyo -Shin이며, 그녀가 그런 멋진 남자...",50,60,0.499451,ENTAILMENT,0.998435,[[tensor(0.7859)]]


코사인 유사도를 이용한 필터링 -> 2


,spoken,written,spoken_length,written_length,blue_score,nli_label,nli_score,cosine_scores
0,"난 지금 행복해요, 세상이 나를 원하고 있죠.",나는 지금 행복합니다. 세상은 나를 원합니다.,25,25,0.400000,ENTAILMENT,0.998424,[[tensor(0.9403)]]
1,안녕하세요 올해가 돼서 매우 기쁘고요. 행복한 올해가 되세요!,"안녕하세요, 나는 올해가되어 매우 기쁩니다. 행복한 한 해를 보냈습니다!",34,40,0.417977,ENTAILMENT,0.987199,[[tensor(0.9039)]]


In [15]:
4 * 100000/10 / 60 / 60

11.11111111111111

In [16]:
df_a = filter(df_a)
df_a

100000


,spoken,written,spoken_length,written_length
0,사과는 잘 씻은 뒤 껍질 채 먹는 게 좋다네요.,사과를 씻고 먹는 것이 낫습니다.,26,18
2,우리가 작성해서 전달해 드려야 하나요?,우리는 쓰고 전달해야합니까?,21,15
4,그는 틀림없이 꽃 가게에 들렸을 겁니다.,그는 꽃 가게에 있었을 것입니다.,22,18
8,"아래 그림의 bga 부분에 대해서는, nsmd에서 smd로 존재하는 것과 같은 방식...",아래 그림의 BGA 부분은 NSMD의 SMD와 같은 방식으로 계속 변경됩니다.,63,43
9,500달러만 당신들에게 요청할게요.,500 달러에만 물을 것입니다.,19,17
...,...,...,...,...
99989,"그러게, 여기로 들어가면 역으로 연결 안 되어 있으려나?",그래서 여기에 가면 역에 연결할 수 없습니까?,31,25
99990,이곳에 오셔서 지내는 동안 음식은 입에 잘 맞으시나요?,여기에 오면서 음식이 입에 맞습니까?,30,20
99992,"그렇군요, 다음부터는 주문할 때 미리 말씀해주세요.","예, 주문할 때 미리 알려주세요.",28,18
99994,"관람 가능한 표를 가지고 있는데, 편할 때 써도 된다고 쓰여있는데 무슨 말인가요?",볼 수있는 티켓이 있습니다.,45,15


문장 길이를 이용한 필터링 -> 60802


/usr/local/lib/python3.8/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.8/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.8/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg

,spoken,written,spoken_length,written_length,blue_score
7,이 호텔은 세계 최악의 호텔로 유명합니다.,이 호텔은 세계 최악의 호텔로 유명합니다.,23,23,1.0
41,우리는 도장을 찍지 않았습니다.,우리는 도장을 찍지 않았습니다.,17,17,1.0
513,그것을 표현하는 다른 단어가 있습니까?,그것을 표현하는 다른 단어가 있습니까?,21,21,1.0
646,바로 옆에 버스 정류장이 있습니다.,바로 옆에 버스 정류장이 있습니다.,19,19,1.0
755,가장 좋아하는 가을 나무는 무엇입니까?,가장 좋아하는 가을 나무는 무엇입니까?,21,21,1.0
...,...,...,...,...,...
94916,실내 일정으로 변경할 계획입니다.,실내 일정으로 변경할 계획입니다.,18,18,1.0
94944,뜨거운 거에 덴 건 아닌 것 같은데 맞나요?,나는 그것이 뜨겁다 고 생각하지 않습니다.,24,23,0.0
95498,"영유아 건강검진 받으러 왔는데, 접수해야 하나요?",나는 유아와 유아를위한 건강 검진을 받기 위해왔다.,27,28,0.0
95930,"아니요, 이 버스는 울산으로 갑니다.","아니요,이 버스는 울산으로갑니다.",20,18,1.0


...

KLUE-NLI를 이용한 필터링를 이용한 필터링 -> 45722


  0%|          | 0/45722 [00:00<?, ?it/s]

,spoken,written,spoken_length,written_length,blue_score,nli_label,nli_score,cosine_scores
1,내가 언제까지 거기 가면 됩니까?,거기에 얼마나 오래 갈 수 있습니까?,18,20,0.250000,ENTAILMENT,0.994652,[[tensor(0.5265)]]
5,"그녀가 좋아하는 가수는 박효신인데, 나는 그녀가 그와 같은 멋진 남자친구가 생기길 ...","그녀가 가장 좋아하는 가수는 Park Hyo -Shin이며, 그녀가 그런 멋진 남자...",50,60,0.499451,ENTAILMENT,0.998435,[[tensor(0.7859)]]
16,만약 아직 자원봉사자가 필요하다면 내가 도와줄게.,여전히 자원 봉사자가 필요하다면 도와 드리겠습니다.,27,28,0.333333,ENTAILMENT,0.998382,[[tensor(0.7386)]]
19,이것은 심장과 폐의 기능을 향상시켜.,이것은 심장과 폐의 기능을 향상시킵니다.,20,22,0.916667,ENTAILMENT,0.998397,[[tensor(0.8700)]]
23,우리 쪽에서 진행 하는 게 맞아요?,우리 편으로 진행하는 것이 옳습니까?,19,20,0.441248,ENTAILMENT,0.998368,[[tensor(0.7864)]]
...,...,...,...,...,...,...,...,...
99985,근데 그건 너무 흔한 이미지라서 좀 식상하네요.,그러나 그것은 매우 일반적인 이미지이므로 약간 지루합니다.,26,32,0.127501,ENTAILMENT,0.998706,[[tensor(0.6993)]]
99991,제 숙소에 외부인이 들어온 거 같아요.,외부인이 내 숙소에 들어갔다고 생각합니다.,21,23,0.600000,ENTAILMENT,0.882948,[[tensor(0.8630)]]
99995,그럼 컨셉을 약간 바꾸는 것도 좋은 방법이겠네요.,그런 다음 개념을 조금 바꾸는 것이 좋습니다.,27,25,0.363636,ENTAILMENT,0.998523,[[tensor(0.7637)]]
99997,4번 게이트로 갔을 때 바로 찾을 수 있나요?,게이트 4에 갔을 때 바로 찾을 수 있습니까?,25,25,0.727273,ENTAILMENT,0.996636,[[tensor(0.8959)]]


코사인 유사도를 이용한 필터링 -> 16840


,spoken,written,spoken_length,written_length,blue_score,nli_label,nli_score,cosine_scores
0,"난 지금 행복해요, 세상이 나를 원하고 있죠.",나는 지금 행복합니다. 세상은 나를 원합니다.,25,25,0.400000,ENTAILMENT,0.998424,[[tensor(0.9403)]]
1,안녕하세요 올해가 돼서 매우 기쁘고요. 행복한 올해가 되세요!,"안녕하세요, 나는 올해가되어 매우 기쁩니다. 행복한 한 해를 보냈습니다!",34,40,0.417977,ENTAILMENT,0.987199,[[tensor(0.9039)]]
2,여행은 예상 못 한 일이 벌어지기 때문에 환상적이에요.,예상치 못한 일이 일어나기 때문에 여행은 환상적입니다.,30,30,0.712278,ENTAILMENT,0.998577,[[tensor(0.9700)]]
3,미국이나 영국에서는 자신을 직접 소개하는 것은 바람직하지 않습니다.,미국과 영국에서는 직접 자신을 소개하는 것이 바람직하지 않습니다.,37,36,0.857143,ENTAILMENT,0.998678,[[tensor(0.9854)]]
4,그는 사우디에서 8개월간 일하고 있습니다.,그는 사우디 아라비아에서 8 개월 동안 일하고 있습니다.,23,31,0.747082,ENTAILMENT,0.998589,[[tensor(0.9620)]]
...,...,...,...,...,...,...,...,...
16835,다른 학교에 가서 수업을 듣는 것입니다. 강의시간표에서 확인할 수 있어요.,다른 학교에 가서 수업을 듣습니다. 강의 일정에서 확인할 수 있습니다.,41,39,0.722222,ENTAILMENT,0.998725,[[tensor(0.9195)]]
16836,할인을 받을 방법이 있을까요?,할인을받을 수있는 방법이 있습니까?,16,19,0.644123,ENTAILMENT,0.996271,[[tensor(0.9833)]]
16837,실례하지만 수영시설이 어디 있나요?,실례하지만 수영 시설은 어디에 있습니까?,19,22,0.600000,ENTAILMENT,0.997817,[[tensor(0.9664)]]
16838,저희는 여기 맥주가 그렇게 맛있다고 들었는데요.,우리는 여기서 맥주가 너무 맛있다고 들었습니다.,26,26,0.596560,ENTAILMENT,0.998104,[[tensor(0.9194)]]


In [18]:
df_a.to_csv('output/df_spoken-written_a_raw.csv')

In [30]:
def filter2(df):
    mask = df['spoken'].str.contains('^.*(니다.)$', regex=True, na=False)
    display(df[mask])
    df = df[-mask]

    mask = df['written'].str.contains('^.*(요.)$', regex=True, na=False)
    display(df[mask])
    df = df[-mask]

    return df

In [32]:
df_a = filter2(df_a)

<ipython-input-30-5c1e4ff06b50>:2: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  mask = df['spoken'].str.contains('^.*(니다.)$', regex=True, na=False)


,spoken,written,spoken_length,written_length,blue_score,nli_label,nli_score,cosine_scores
3,미국이나 영국에서는 자신을 직접 소개하는 것은 바람직하지 않습니다.,미국과 영국에서는 직접 자신을 소개하는 것이 바람직하지 않습니다.,37,36,0.857143,ENTAILMENT,0.998678,[[tensor(0.9854)]]
4,그는 사우디에서 8개월간 일하고 있습니다.,그는 사우디 아라비아에서 8 개월 동안 일하고 있습니다.,23,31,0.747082,ENTAILMENT,0.998589,[[tensor(0.9620)]]
10,우리는 그들의 심의 결과를 웹사이트에서 확인합니다.,우리는 웹 사이트에서 그들의 심의를 확인합니다.,28,26,0.846154,ENTAILMENT,0.998661,[[tensor(0.9355)]]
12,남자는 수영장을 만들기로 하였습니다.,그 남자는 수영장을 만들기로 결정했습니다.,20,23,0.681451,ENTAILMENT,0.998479,[[tensor(0.9627)]]
14,재단 후 이미지로 다시 업로드 합니다.,재단 후 이미지에 다시 업로드하십시오.,21,21,0.750000,ENTAILMENT,0.998633,[[tensor(0.9719)]]
...,...,...,...,...,...,...,...,...
16825,박물관의 기념품 가게에서 최근에 더 좋은 제품들을 보았습니다.,박물관 기념품 가게에서 최근에 더 나은 제품을 보았습니다.,34,32,0.785714,ENTAILMENT,0.998497,[[tensor(0.9381)]]
16827,오늘 밤 영화 상영시간표를 보고 싶습니다.,오늘 밤 영화 상영 일정을보고 싶습니다.,23,22,0.777778,ENTAILMENT,0.998345,[[tensor(0.9184)]]
16828,선글라스는 30달러로 총 180달러입니다.,선글라스는 $ 30에 $ 180입니다.,23,21,0.441248,ENTAILMENT,0.995804,[[tensor(0.9207)]]
16830,이번 프레젠테이션은 영어로 해야 할 것 같습니다.,이 프레젠테이션은 영어로 이루어져야한다고 생각합니다.,27,29,0.615385,ENTAILMENT,0.998603,[[tensor(0.9161)]]


<ipython-input-30-5c1e4ff06b50>:6: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  mask = df['written'].str.contains('^.*(요.)$', regex=True, na=False)


,spoken,written,spoken_length,written_length,blue_score,nli_label,nli_score,cosine_scores
58,우즈베키스탄에 언제 가세요?,우즈베키스탄에 언제 갈 건가요?,15,17,0.705401,ENTAILMENT,0.998506,[[tensor(0.9483)]]
75,버스 타고 가는 건 시간이 오래 걸려요?,버스를 타는 데 시간이 오래 걸리나요?,22,21,0.555556,ENTAILMENT,0.998262,[[tensor(0.9567)]]
100,교환된 제품은 언제 돌아옵니까?,교환 된 제품은 언제 돌아올까요?,17,18,0.743038,ENTAILMENT,0.997109,[[tensor(0.9258)]]
127,실례지만 혹시 질문 하나 해도 될까요?,실례합니다.하지만 질문을해도 될까요?,21,20,0.497133,ENTAILMENT,0.998451,[[tensor(0.9467)]]
136,내일 날씨는 눈이 조금 올 것 같아요.,내일 날씨가 약간 눈이 될 것 같아요.,21,21,0.700000,ENTAILMENT,0.998625,[[tensor(0.9778)]]
...,...,...,...,...,...,...,...,...
16764,"정말 맛있는 냄새가 나네요, 얼른 먹고 싶어요.",정말 맛있는 냄새가납니다. 빨리 먹고 싶어요.,26,25,0.700000,ENTAILMENT,0.998441,[[tensor(0.9428)]]
16773,지금 당장 화장실을 가고 싶어요.,지금 당장 화장실에 가고 싶어요.,18,18,0.857143,ENTAILMENT,0.997594,[[tensor(0.9885)]]
16784,쌀로 만든 푸딩이 있다고 하던데 사실인가요?,쌀로 만든 푸딩이 있다고합니다.사실인가요?,24,23,0.814354,ENTAILMENT,0.997842,[[tensor(0.9903)]]
16803,"직업이 간호사인데, 무슨 일인지 설명해주세요.",직업은 간호사입니다. 무슨 일이 일어나는지 설명 해주세요.,25,32,0.545455,ENTAILMENT,0.998490,[[tensor(0.9574)]]


In [33]:
df_a.to_csv('output/df_spoken-written_a_raw2.csv')

In [34]:
df_a[['spoken', 'written']].to_csv('output/df_spoken-written_a.csv')

In [35]:
pd.read_csv('output/df_spoken-written_a.csv')

,Unnamed: 0,spoken,written
0,0,"난 지금 행복해요, 세상이 나를 원하고 있죠.",나는 지금 행복합니다. 세상은 나를 원합니다.
1,1,안녕하세요 올해가 돼서 매우 기쁘고요. 행복한 올해가 되세요!,"안녕하세요, 나는 올해가되어 매우 기쁩니다. 행복한 한 해를 보냈습니다!"
2,2,여행은 예상 못 한 일이 벌어지기 때문에 환상적이에요.,예상치 못한 일이 일어나기 때문에 여행은 환상적입니다.
3,5,식당은 적절한 가격의 최상 서비스를 추구해요.,식당은 적절한 가격으로 최고의 서비스를 추구합니다.
4,6,저는 금연 실을 예약했는데 방에서 담배 냄새가 엄청나요.,나는 금연 공간을 예약했지만 방에 담배 냄새는 엄청납니다.
...,...,...,...
10106,16834,지금 우리가 어디쯤 걷고 있는지 확인할 수 있어?,우리가 지금 어디에서 걷고 있는지 볼 수 있습니까?
10107,16835,다른 학교에 가서 수업을 듣는 것입니다. 강의시간표에서 확인할 수 있어요.,다른 학교에 가서 수업을 듣습니다. 강의 일정에서 확인할 수 있습니다.
10108,16836,할인을 받을 방법이 있을까요?,할인을받을 수있는 방법이 있습니까?
10109,16837,실례하지만 수영시설이 어디 있나요?,실례하지만 수영 시설은 어디에 있습니까?


# References

- https://drive.google.com/file/d/14SnyQH0GUkPYHBuz52OLTwcjKmbsGlJF/view?usp=share_link
- http://incredible.ai/nlp/2020/02/29/BLEU/
- https://github.com/Huffon/klue-transformers-tutorial/blob/master/natural_language_inference.ipynb
- https://huggingface.co/ddobokki/klue-roberta-small-nli-sts
- https://www.sbert.net/docs/usage/semantic_textual_similarity.html